In [5]:
from models.vct0 import VCT0Prefix
import torch
from datasets import load_dataset
from transformers import AutoTokenizer

In [6]:
args = {
    "prefix_length": 10,
    "prefix_size": 768,
    "mapping_type": "mlp",
    "model_version": "bigscience/T0_3B"
}
model = VCT0Prefix(**args)
checkpoint = torch.load("/home/rsa39/rds/rds-mlmi-2020-21-xyBFuSj0hm0/MLMI.2021-22/rsa39/project/RAVQA/Experiments/VC-T0_3B-Conceptual-Captions-MLP-Prefix10/train/saved_model/model_04.ckpt", map_location=torch.device('cpu'))
# checkpoint = torch.load("/home/rsa39/rds/rds-mlmi-2020-21-xyBFuSj0hm0/MLMI.2021-22/rsa39/project/RAVQA/Experiments/VC-T0pp-Conceptual-Captions-MLP-Prefix5/train/saved_model/model_01.ckpt", map_location=torch.device('cpu'))
state_dict = {key[6:]: value for key, value in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B")



 Using MLP 


MLP(
  (model): Sequential(
    (0): Linear(in_features=768, out_features=10240, bias=True)
    (1): Tanh()
    (2): Linear(in_features=10240, out_features=20480, bias=True)
  )
)


In [7]:
con_caps = load_dataset(
    "parquet",
    data_files={
        "val": "/home/rsa39/rds/rds-mlmi-2020-21-xyBFuSj0hm0/MLMI.2021-22/rsa39/project/RAVQA/data/conceptual_captions/pre-extracted-features/conceptual_captions_ViT-L_14@336px_validation.parquet",
    },
)

Using custom data configuration default-f878f8eac05c77e9
Reusing dataset parquet (/home/rsa39/.cache/huggingface/datasets/parquet/default-f878f8eac05c77e9/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
images_of_interest = [
    "https://i.pinimg.com/736x/b3/1d/79/b31d7983262164d4025e2f95eb52172d--shakespeare-characters-shakespeare-love.jpg",
    "https://ak0.picdn.net/shutterstock/videos/5526380/thumb/1.jpg",
    "https://thumb1.shutterstock.com/image-photo/stock-photo-pair-of-new-bright-orange-modern-sneakers-isolated-on-a-white-background-450w-598949420.jpg",
    "http://0345ed7.netsolhost.com/images/no_drones.jpg",
    "http://c8.alamy.com/comp/KFK4TJ/zoo-keeper-feeding-apples-to-an-elephant-in-captivity-spain-KFK4TJ.jpg"
]

text_list = [
    "Summarize: <extra_id_0>",
    # "Reorder the words in this sentence: <extra_id_0>",
    # "Write a dialogue that matches this summary: <extra_id_0>",
    # "Write an expanded news article with plausible details from the following summary: <extra_id_0>",
    # "We have the sentence: <extra_id_0>; Extract all the key concepts:",
    # "Humans can easily string together abstract concepts to form a coherent sentence. For example, with the concepts <extra_id_0>, a simple sentence can be",
    # "Ignoring the order of the concepts: <extra_id_0>; Generate a sentence with all the concepts:",
    # "Extract the answer to the question from the following context. Question: Is the cat wearing a crown? Context: <extra_id_0>",
    # "Read the following context and choose the best option to answer the question.\nContext: <extra_id_0>\nQuestion: what does the sign say? \nOptions:\nA. Unattended vehicles will be towed at the owner's expense\nB. This is ross street\nC. No camping\nD. a cat lives here"
]

In [28]:
decoder_text_input = "<pad> A picture of"
decoder_input = tokenizer.encode(decoder_text_input, return_tensors="pt")[:, :-1]

In [29]:
for text in text_list:
    print("\n\n" + text + "\n\n")
    text_inputs = tokenizer(text, return_tensors="pt")
    for entry in con_caps["val"]:
        if entry['image_url'][0] in images_of_interest:
            print(entry["caption"][0])
            clip_embedding = entry["clip_embeddings"]
            torch_clip_embedding = torch.tensor(clip_embedding).view(1, 1, -1)
            print(tokenizer.decode(model.generate(prefix=torch_clip_embedding, max_length=50)[0]))
            print(tokenizer.decode(model.generate(prefix=torch_clip_embedding, question_tokens=text_inputs.input_ids, question_mask=text_inputs.attention_mask, max_length=50)[0]))
            print(tokenizer.decode(model.generate(prefix=torch_clip_embedding, question_tokens=text_inputs.input_ids, question_mask=text_inputs.attention_mask, decoder_input_ids=decoder_input, max_length=50)[0]))



We have the sentence: <extra_id_0>; Extract all the key concepts:


person feeding apples to an elephant in captivity.
a young elephant with a bottle of water.</s>
a skull and cross bones pirate flag blows in the wind.
a pirate ship with a flag flying.</s>
combines famous scenes from plays with the internet 's favorite animal.
a cat with a crown.</s>
picture of presidents with sign in the foreground.
a man with a gun pointing at a sign.</s>
